In [ ]:
import matplotlib.pyplot as plt
import pandas as pd
import json
from src.pokemon import Pokemon, PokemonFactory, StatusEffect
from src.catching import attempt_catch

In [ ]:
available_pokemon = []
with open("pokemon.json", "r") as file:
    pokemon_db = json.load(file)
    available_pokemon = [key for key in pokemon_db]

available_pokemon

In [ ]:
with open("ejercicio1b-config.json", "r") as file:
    config = json.load(file)

ITERS = config["iterations"]
LEVEL = config["level"]
STATUS = StatusEffect[config["status"]]
HP_PERCENTAGE = config["hp_percentage"]
POKEBALLS = config["pokeballs"]
POKEBALL_COMPARE = config["pokeball_compare"]
(ITERS, LEVEL, STATUS, HP_PERCENTAGE, POKEBALLS, POKEBALL_COMPARE)

In [ ]:
factory = PokemonFactory("pokemon.json")

def test_pokemon(pokemon_name):
    pokemon = factory.create(pokemon_name, LEVEL, STATUS, HP_PERCENTAGE)
    df = pd.DataFrame(columns=["pokeball", "success"])
    for pokeball in POKEBALLS:
        for _ in range(ITERS):
            attempt, rate = attempt_catch(pokemon, pokeball)
            new_row = [pokeball, attempt]
            df.loc[len(df)] = new_row
    
    df = df.groupby("pokeball").agg({"success": ["mean", "std"]}).reset_index()
    df.columns = df.columns.map("_".join)
    
    # Find the common pokeball's mean success rate in the dataset
    common_pokeball_mean = 0
    for i in range(len(df)):
        if (df.loc[i]["pokeball_"] == POKEBALL_COMPARE):
            common_pokeball_mean = df.loc[i]["success_mean"]

    df["relative_mean"] = df.success_mean / common_pokeball_mean
    df["relative_std"] = df.success_std / common_pokeball_mean
    return df

In [ ]:
def make_graph(df):
    # Crear una figura y un eje
    fig, ax = plt.subplots(figsize=(10, 6))

    # Crear las barras del gráfico
    x = df["pokeball_"]
    y = df["relative_mean"]
    #error = df["relative_std"]
    #ax.bar(x, y, yerr=error, capsize=5)
    ax.bar(x, y, capsize=5)

    # Configurar el eje x
    ax.set_xticks(range(len(x)))
    ax.set_xticklabels(x, rotation=45, ha="right")
    ax.set_xlabel("Pokebola")

    # Configurar el eje y
    ax.set_ylabel("Eficiencia relativa")

    # Añadir título
    ax.set_title("Promedios de eficiencia relativa de las pokebolas respectos a la pokebola común")

    # Mostrar el gráfico
    return plt.show()

In [ ]:
caterpie_df = test_pokemon("caterpie")
caterpie_df

In [ ]:
make_graph(caterpie_df)

In [ ]:
jolteon_df = test_pokemon("jolteon")
make_graph(jolteon_df)

In [ ]:
snorlax_df = test_pokemon("snorlax")
make_graph(snorlax_df)

In [ ]:
onix_df = test_pokemon("onix")
make_graph(onix_df)

In [ ]:
mewtwo_df = test_pokemon("mewtwo")
make_graph(mewtwo_df)